##### IMPORTAR AS PLANILHAS

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

##### IMPORTANDO PLANILHA

In [ ]:
path_iw = "data/util/os/"

path_iw = ajuste_path(path_iw)

df_iw47 = pd.read_csv(path_iw + "IW47_Executadas_preparado.csv")

df_iw47.columns

In [ ]:
colunas_tipo_atividade = ["Txt.breve operação", "Denominação TAM"]

In [ ]:
df_iw47["Data fim"] = pd.to_datetime(df_iw47["Data fim"])
df_iw47["Data inicio"] = pd.to_datetime(df_iw47["Data inicio"])
df_iw47["Local de instalação"] = df_iw47["Local de instalação"].astype(str)

##### CRIANDO DATAFRAME SEM INCLUIR DATAS FINAIS ANTERIORES A DATAS INICIAIS

In [ ]:
# Removendo linhas onde a data final é superior a data inicial
df_sem_linhas_irregulares = df_iw47.loc[df_iw47["Data fim"]
                                        >= df_iw47["Data inicio"]]

num_linhas_removidas = df_iw47.shape[0] - df_sem_linhas_irregulares.shape[0]
print("Linhas removidas:", num_linhas_removidas)

##### FUNÇÃO QUE ADD COLUNA DE HH POR MES E QUANTIDADE DE MESES PASSADOS DURANTE A OS

In [ ]:
# Soma dos meses, em funcao da quantidade de anos que se passaram e meses, entre o inicio e fim da OS
df = df_sem_linhas_irregulares

df["Meses"] = (
    (df["Data fim"].dt.year - df["Data inicio"].dt.year) * 12
    + (df["Data fim"].dt.month - df["Data inicio"].dt.month)
) + 1
df["HH por mes"] = df["HH final"] / df["Meses"]

##### ADICIONANDO DURAÇÃO DA OS ANTES DE AGRUPAR PARA MAIOR ACURÁCIA

In [ ]:
import numpy as np

# Supondo que as colunas 'Data inicio' e 'Data fim' já estão no formato datetime64
df['Data inicio'] = pd.to_datetime(df['Data inicio'])
df['Data fim'] = pd.to_datetime(df['Data fim'])

# Criar uma condição para verificar se a Data fim é 2020 ou mais recente e Data inicio é 2019 ou anterior
condicao = (df['Data fim'] >= pd.Timestamp('2020-01-01')) & (df['Data inicio'] < pd.Timestamp('2020-01-01'))

# Aplicar a condição para ajustar a Data inicio
df.loc[condicao, 'Data inicio'] = pd.Timestamp('2020-01-01')

# Calcular a duração em dias úteis e multiplicar por 8 para horas
df['Duração'] = (np.busday_count(df['Data inicio'].values.astype('datetime64[D]'),
                                 df['Data fim'].values.astype('datetime64[D]')) + 1)*8  # 8 horas por dia

# Calcular a quantidade de meses após 2020 visto que mudamos a data de inicio
df['Meses após 2020'] = (df['Data fim'].dt.year - df['Data inicio'].dt.year) * 12 + (df['Data fim'].dt.month - df['Data inicio'].dt.month + 1) 

# Calcular a duração em meses
df['Duração'] = df['Duração']/ df['Meses após 2020']

##### FUNCAO QUE AGRUPA HH POR ANO, MES, LOCAL E TIPO DE ATIVIDADE

In [ ]:
df["Data inicio"] = df["Data inicio"].apply(lambda x: x.replace(day=1))
df["Data fim"] = df["Data fim"].apply(lambda x: x.replace(day=1))

In [ ]:
# Gerar o espaço de tempo
espaco_de_tempo = pd.date_range(start='2020-01-01', end=pd.Timestamp.now(), freq="MS")
espaco_de_tempo = pd.DataFrame({'merge': [1]*len(espaco_de_tempo), 'Ano Mes': espaco_de_tempo})

# Marcar operações que duram mais de um mês
df.loc[df['Meses'] > 1, 'dura mais de mes'] = 1

# Manter apenas as colunas necessárias
colunas_a_manter = [
    'Local de instalação',
    'HH por mes',
    'Meses',
    'dura mais de mes',
    'Data inicio',
    'Data fim'
]
colunas_a_manter.extend(colunas_tipo_atividade)
df = df[colunas_a_manter]

# Expansão das operações que duram mais de um mês
df_expandido = pd.merge(df, espaco_de_tempo, left_on='dura mais de mes', right_on='merge', how='left').drop(columns=['merge'])

# Tratamento das colunas de data para garantir o correto agrupamento
df_expandido['dura mais de mes'] = df_expandido['dura mais de mes'].fillna(0)
df_expandido.loc[df_expandido['dura mais de mes'] == 0, 'Ano Mes'] = df_expandido['Data fim']
df_expandido['Ano Mes'] = df_expandido['Ano Mes'].dt.to_period('M')
df_expandido['Data inicio'] = df_expandido['Data inicio'].dt.to_period('M')
df_expandido['Data fim'] = df_expandido['Data fim'].dt.to_period('M')

# Eliminar linhas onde a operação não estava ativa
df_expandido = df_expandido[(df_expandido['Ano Mes'] >= df_expandido['Data inicio']) &
                            (df_expandido['Ano Mes'] <= df_expandido['Data fim'])]

# Calcular a duração em dias específica para cada mês
df_expandido['inicio_mes_corrente'] = df_expandido['Ano Mes'].dt.to_timestamp()
df_expandido['fim_mes_corrente'] = df_expandido['Ano Mes'].dt.to_timestamp('M') + pd.offsets.MonthEnd(0)

df_expandido['data_inicio_real'] = np.maximum(df_expandido['Data inicio'].dt.to_timestamp(), df_expandido['inicio_mes_corrente'])
df_expandido['data_fim_real'] = np.minimum(df_expandido['Data fim'].dt.to_timestamp(), df_expandido['fim_mes_corrente'])

# Conta os dias úteis entre as datas ajustadas
df_expandido['duracao_dias'] = (np.busday_count(df_expandido['data_inicio_real'].values.astype('datetime64[D]'),
                                               df_expandido['data_fim_real'].values.astype('datetime64[D]')) + 1)*8 # 8 horas por dia

# Manter apenas as colunas necessárias para o agrupamento final
colunas_a_manter = ['Ano Mes', 'Local de instalação', 'HH por mes', 'duracao_dias']
colunas_a_manter.extend(colunas_tipo_atividade)
df_expandido = df_expandido[colunas_a_manter]

# Para cada categoria de atividade, criar uma nova coluna para armazenar o HH dessa categoria
for coluna_tipo_atividade in colunas_tipo_atividade:
    for atividade in df_expandido[coluna_tipo_atividade].unique():
        if "HH de " + atividade in df_expandido.columns:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "HH de " + atividade + " de " + coluna_tipo_atividade] = df_expandido["HH por mes"]
        else:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "HH de " + atividade] = df_expandido["HH por mes"]

# Limpa colunas desnecessárias após o agrupamento
df_agrupado = df_expandido.groupby(['Ano Mes', 'Local de instalação']).sum().reset_index()
df_agrupado.drop(columns=colunas_tipo_atividade, inplace=True)
df_agrupado.rename(columns={'HH por mes': 'HH total'}, inplace=True)

# Filtra para considerar apenas registros a partir de 2020-01
df_agrupado = df_agrupado[df_agrupado['Ano Mes'] >= '2020-01']


##### TRATAMENTO FINAL PARA PODER EXPORTAR A TABELA

In [ ]:
df_agrupado = df_agrupado.sort_values(
    by=["Ano Mes", "Local de instalação"]).reset_index(drop=True)

df_agrupado.fillna(0, inplace=True)

##### EXPORTANDO PLANILHA

In [ ]:
df_agrupado.to_csv(path_iw + "os_treinamento.csv", index=False)